In [1]:
# ============================================
# train_optimal_load_model.py
# RandomForest + Optuna (200 trials, k=5 estratificado por Product_ID)
# Dataset base (vuelo × producto), ignora Crew_Feedback y Unit_Cost
# Feature engineering: Spec_per_Passenger, Spec_x_Passengers, IsWeekend, Quarter,
#                      Avg_Consumed_Product, Avg_Returned_Product
# Guarda: best_model.pkl, X_columns.pkl, model_metadata.pkl, model_fit_plot.png
# ============================================

import os
import joblib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# -------------------------
# 1) Cargar datos
# -------------------------
PATH = "/Users/luisgarcia/Documents/LuisCoding/Septimo/2 y 3 periodo/HACKMTY/hackmty2025-gategroup/consumption-prediction/data/ConsumptionPrediction_Dataset_v1.xlsx"  # ajusta si es necesario
if not os.path.exists(PATH):
    raise FileNotFoundError(f"No se encontró {PATH} en el directorio actual.")
df = pd.read_excel(PATH, sheet_name=0)
df = df.replace({True: 1, False: 0})

# Columnas esperadas (de tu dataset)
expected = [
    "Flight_ID","Origin","Date","Flight_Type","Service_Type",
    "Passenger_Count","Product_ID","Product_Name",
    "Standard_Specification_Qty","Quantity_Returned","Quantity_Consumed","Unit_Cost","Crew_Feedback"
]
missing = [c for c in expected if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas esperadas: {missing}")

# -------------------------
# 2) Limpieza y filtros
# -------------------------
# Ignorar columnas no usadas
df = df.drop(columns=["Crew_Feedback"], errors="ignore")

# Parseo de fecha y derivados
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df = df.dropna(subset=["Date"])
df["Month"] = df["Date"].dt.month
df["DayOfWeek"] = df["Date"].dt.dayofweek  # 0=Lun … 6=Dom
df["IsWeekend"] = df["DayOfWeek"].isin([5, 6]).astype(int)
df["Quarter"] = df["Month"].apply(lambda x: (int(x)-1)//3 + 1)

# % devuelto (para definir el target)
eps = 1e-9
df["Percentage_Returned"] = df["Quantity_Returned"] / (df["Standard_Specification_Qty"] + eps)

# Target conservador básico: consumo + margen por promedio global del retorno
avg_pr = df["Percentage_Returned"].replace([np.inf, -np.inf], np.nan).dropna().mean()
avg_pr = 0.0 if pd.isna(avg_pr) else float(avg_pr)
df["Optimal_Load"] = df["Quantity_Consumed"] + avg_pr * df["Standard_Specification_Qty"]

# -------------------------
# 3) Features por producto (históricos)
# -------------------------
# Promedios históricos por producto (no son fuga: son medias históricas generales)
prod_stats = (
    df.groupby("Product_ID")[["Quantity_Consumed", "Quantity_Returned"]]
      .mean()
      .rename(columns={"Quantity_Consumed":"Avg_Consumed_Product",
                       "Quantity_Returned":"Avg_Returned_Product"})
)
df = df.merge(prod_stats, on="Product_ID", how="left")

# Interacciones
df["Spec_per_Passenger"] = df["Standard_Specification_Qty"] / (df["Passenger_Count"] + eps)
df["Spec_x_Passengers"]  = df["Standard_Specification_Qty"] * df["Passenger_Count"]

# -------------------------
# 4) Diseño de features (sin fugas)
# -------------------------
# Excluir post-vuelo y columnas no informativas / no usadas
drop_cols = {
    "Quantity_Consumed","Quantity_Returned","Percentage_Returned",
    "Optimal_Load","Date","Product_Name","Flight_ID","Unit_Cost"  # fuera Unit_Cost
}

cat_cols = ["Origin","Flight_Type","Service_Type","Product_ID"]
num_cols = [
    "Passenger_Count","Standard_Specification_Qty",
    "Month","DayOfWeek","IsWeekend","Quarter",
    "Avg_Consumed_Product","Avg_Returned_Product",
    "Spec_per_Passenger","Spec_x_Passengers",
]

for c in cat_cols + num_cols:
    if c not in df.columns:
        raise ValueError(f"Falta columna para features: {c}")

X = pd.concat(
    [df[num_cols].reset_index(drop=True),
     pd.get_dummies(df[cat_cols], prefix=cat_cols, drop_first=False).reset_index(drop=True)],
    axis=1
)
y = df["Optimal_Load"].astype(float)

# Mantener una serie con Product_ID (para estratificación)
prod_labels = df["Product_ID"].astype(str).reset_index(drop=True)

# -------------------------
# 5) Split train/test
# -------------------------
X_train, X_test, y_train, y_test, prod_train, prod_test = train_test_split(
    X, y, prod_labels, test_size=0.20, random_state=42, stratify=prod_labels
)

# -------------------------
# 6) Optuna: RF con CV estratificada por Product_ID
# -------------------------
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 600, 3000),
        "max_depth": trial.suggest_int("max_depth", 8, 60),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 40),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
    }
    model = RandomForestRegressor(**params, random_state=42, n_jobs=-1)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    mses = []
    for tr_idx, val_idx in skf.split(X_train, prod_train):
        X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        model.fit(X_tr, y_tr)
        pred = model.predict(X_val)
        mses.append(mean_squared_error(y_val, pred))
    return float(np.mean(mses))

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, n_jobs=-1, show_progress_bar=True)

print("\n✅ Best hyperparameters:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

best_model = RandomForestRegressor(**study.best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

# -------------------------
# 7) Evaluación
# -------------------------
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)
print("\n==== Final Test Metrics ====")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²:  {r2:.4f}")

# Plot y_test vs y_pred
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
lo = float(min(y_test.min(), y_pred.min()))
hi = float(max(y_test.max(), y_pred.max()))
plt.plot([lo, hi], [lo, hi], "--", color="gray")
plt.xlabel("Real (y_test)")
plt.ylabel("Predicho (y_pred)")
plt.title("Predicción vs Real - Optimal_Load (RF mejorado)")
plt.tight_layout()
plt.savefig("model_fit_plot.png")
plt.close()

# -------------------------
# 8) Guardar artefactos y metadatos
# -------------------------
joblib.dump(best_model, "best_model.pkl")
joblib.dump(X.columns.tolist(), "X_columns.pkl")

# Opciones para dashboard
origin_opts = sorted(df["Origin"].dropna().astype(str).unique().tolist())
ftype_opts  = sorted(df["Flight_Type"].dropna().astype(str).unique().tolist())
stype_opts  = sorted(df["Service_Type"].dropna().astype(str).unique().tolist())
product_opts = sorted(df["Product_ID"].dropna().astype(str).unique().tolist())

# Mapa Product_ID → Product_Name
pid_name_map = (
    df[["Product_ID","Product_Name"]]
      .dropna().drop_duplicates(subset=["Product_ID"])
      .astype({"Product_ID": str})
      .set_index("Product_ID")["Product_Name"].astype(str).to_dict()
)

# Medianas de qty por producto (para defaults)
per_product_qty_median = {
    str(pid): float(grp["Standard_Specification_Qty"].median())
    for pid, grp in df.groupby("Product_ID")
}

# Medias históricas por producto (para construir features en el dashboard)
avg_by_product = (
    df.groupby("Product_ID")[["Avg_Consumed_Product", "Avg_Returned_Product"]]
      .mean()
      .astype(float)
      .rename_axis("Product_ID")
      .reset_index()
)
avg_map = {
    str(row["Product_ID"]): {
        "Avg_Consumed_Product": float(row["Avg_Consumed_Product"]),
        "Avg_Returned_Product": float(row["Avg_Returned_Product"]),
    }
    for _, row in avg_by_product.iterrows()
}

metadata = {
    "cat_cols": cat_cols,
    "num_cols": num_cols,
    "onehot_prefixes": {c: f"{c}_" for c in cat_cols},
    "origin_options": origin_opts,
    "flight_type_options": ftype_opts,
    "service_type_options": stype_opts,
    "product_id_options": product_opts,
    "product_id_to_name": pid_name_map,
    "per_product_qty_median": per_product_qty_median,
    "avg_by_product": avg_map,  # para features históricas
}

joblib.dump(metadata, "model_metadata.pkl")

print("\n📦 Artefactos guardados:")
print("- best_model.pkl")
print("- X_columns.pkl")
print("- model_metadata.pkl")
print("- model_fit_plot.png")

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-26 02:56:06,054] A new study created in memory with name: no-name-b41e850b-5d6a-405e-82d9-a57e26c1e3b3
Best trial: 0. Best value: 218.13:   1%|          | 1/100 [00:12<21:25, 12.99s/it]

[I 2025-10-26 02:56:18,939] Trial 0 finished with value: 218.13026734954275 and parameters: {'n_estimators': 706, 'max_depth': 36, 'min_samples_split': 27, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 218.13026734954275.


Best trial: 1. Best value: 204.14:   2%|▏         | 2/100 [00:14<10:31,  6.44s/it]

[I 2025-10-26 02:56:20,812] Trial 1 finished with value: 204.1395300748326 and parameters: {'n_estimators': 859, 'max_depth': 44, 'min_samples_split': 27, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 1 with value: 204.1395300748326.


Best trial: 4. Best value: 194.519:   3%|▎         | 3/100 [00:26<14:12,  8.79s/it]

...

[I 2025-10-26 03:02:39,463] Trial 97 finished with value: 187.9017493588188 and parameters: {'n_estimators': 1528, 'max_depth': 47, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 63 with value: 186.49830472094314.
[I 2025-10-26 03:02:39,533] Trial 96 finished with value: 187.76498503623654 and parameters: {'n_estimators': 1669, 'max_depth': 44, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 63 with value: 186.49830472094314.


Best trial: 63. Best value: 186.498:  99%|█████████▉| 99/100 [06:33<00:01,  1.29s/it]

[I 2025-10-26 03:02:39,808] Trial 98 finished with value: 187.80317133819662 and parameters: {'n_estimators': 1640, 'max_depth': 49, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 63 with value: 186.49830472094314.


Best trial: 63. Best value: 186.498: 100%|██████████| 100/100 [06:34<00:00,  3.94s/it]


[I 2025-10-26 03:02:40,164] Trial 99 finished with value: 187.73653470442565 and parameters: {'n_estimators': 1684, 'max_depth': 47, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 63 with value: 186.49830472094314.

✅ Best hyperparameters:
n_estimators: 2051
max_depth: 13
min_samples_split: 5
min_samples_leaf: 4
max_features: sqrt
bootstrap: False

==== Final Test Metrics ====
MSE: 205.1958
MAE: 10.2891
R²:  0.9372

📦 Artefactos guardados:
- best_model.pkl
- X_columns.pkl
- model_metadata.pkl
- model_fit_plot.png
